In [1]:
import netCDF4
import itertools

from tqdm import tqdm

Nx = 512
Ny = 512
Nz = 601
Na = 5
Ns = Nx * Ny

In [11]:
import itertools

from scipy import interpolate

import sys

def find_interpol_range(tau, Nz, Ns):

    thick = 1e+20
    thinn = 1e-20

    for k in range(Ns):

        opt_thick_end = tau[Nz - 1, k]
        opt_thinn_end = tau[0, k]

        if opt_thinn_end > thinn: thinn = opt_thinn_end
        
        if opt_thick_end < thick: thick = opt_thick_end
    
    return np.log10(thinn), np.log10(thick)

def find_shortest_delta(tau, Nz, Ns):
    
    tau_1d = np.log10(tau.T).reshape(Nz * Ns)
    
    delta = np.abs(np.diff(tau_1d))
    
    idx_min = np.argmin(np.abs(delta - 0.001))
            
    return delta[idx_min]

def const_interpol_grid(tau, Nz, Ns):
    
    thin, thick = find_interpol_range(tau, Nz, Ns)
    delta =       find_shortest_delta(tau, Nz, Ns)
    
    Ng = int(np.floor(np.abs(thick - thin) / delta))
    
    grid = np.zeros(Ng)
    
    grid[0] = thin
    
    for i in range(Ng - 1): grid[i + 1] = grid[i] + delta
    
    return grid

def interpol_and_average(i, target, tau, T, igrid, Ns):
    
    T_mean = np.zeros(len(igrid))
    
    for k in tqdm(range(Ns), desc = 'interpolate and average ' + target):
    
        f = interpolate.interp1d(np.log10(tau[:, k]), T[:, k])
        
        Ti = f(igrid)
        
        T_mean += Ti
        
    T_mean /= Ns
    
    return T_mean

In [12]:
mu_id = ['10', '8', '6', '4', '2']

T = np.zeros((Na, Nz, Ny * Nx))
p = np.zeros((Na, Nz, Ny * Nx))
d = np.zeros((Na, Nz, Ny * Nx))
t = np.zeros((Na, Nz, Ny * Nx))

T_ssd = np.zeros((Na, Nz, Ns))
p_ssd = np.zeros((Na, Nz, Ns))
d_ssd = np.zeros((Na, Nz, Ns))
t_ssd = np.zeros((Na, Nz, Ns))

T_300 = np.zeros((Na, Nz, Ns))
p_300 = np.zeros((Na, Nz, Ns))
d_300 = np.zeros((Na, Nz, Ns))
t_300 = np.zeros((Na, Nz, Ns))

grid_ssd = np.ones((Na, 110000)) * np.nan
grid_300 = np.ones((Na, 110000)) * np.nan

Ti_ssd = np.ones((Na, 110000)) * np.nan
pi_ssd = np.ones((Na, 110000)) * np.nan
di_ssd = np.ones((Na, 110000)) * np.nan

Ti_300 = np.ones((Na, 110000)) * np.nan
pi_300 = np.ones((Na, 110000)) * np.nan
di_300 = np.ones((Na, 110000)) * np.nan

for i in range(Na):
    
    print("---------------")
    print('mu = ', mu_id[i])
    print("---------------")
    
    print('SSD cube')
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    print('reading tau...')
    t[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/taugrid.222809.nc.1')['tau']).reshape(Nz, Ny * Nx)
    
    print('sampling tau...')
    tau = t[i, :, sample].T
    
    grid = const_interpol_grid(tau, Nz, Ns)
    
    grid_ssd[i, : len(grid)] = 10.0**grid
    
    print('reading temperature...')
    T[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/T_onTau.222809.nc.1')['T']).reshape(Nz, Ny * Nx)
    print('reading pressure...')
    p[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/P_onTau.222809.nc.1')['P']).reshape(Nz, Ny * Nx)
    print('reading density...')
    d[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/rho_onTau.222809.nc.1')['R']).reshape(Nz, Ny * Nx)
    
    T_mean = interpol_and_average(i, 'SSD temp', tau, T[i, :, sample].T, grid, Ns)
    p_mean = interpol_and_average(i, 'SSD pres', tau, p[i, :, sample].T, grid, Ns)
    d_mean = interpol_and_average(i, 'SSD dens', tau, d[i, :, sample].T, grid, Ns)

    Ti_ssd[i, : len(T_mean)] = T_mean
    pi_ssd[i, : len(p_mean)] = p_mean
    di_ssd[i, : len(d_mean)] = d_mean
    
    print('300G cube')
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)

    print('reading tau...')
    t[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/taugrid.627321.nc.1')['tau']).reshape(Nz, Ny * Nx)
    
    print('sampling tau...')
    tau = t[i, :, sample].T
    
    grid = const_interpol_grid(tau, Nz, Ns)
    
    grid_300[i, : len(grid)] = 10.0**grid
    
    print('reading temperature...')
    T[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/T_onTau.627321.nc.1')['T']).reshape(Nz, Ny * Nx)
    print('reading pressure...')
    p[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/P_onTau.627321.nc.1')['P']).reshape(Nz, Ny * Nx)
    print('reading density...')
    d[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/rho_onTau.627321.nc.1')['R']).reshape(Nz, Ny * Nx)
    
    T_mean = interpol_and_average(i, '300G temp', tau, T[i, :, sample].T, grid, Ns)
    p_mean = interpol_and_average(i, '300G pres', tau, p[i, :, sample].T, grid, Ns)
    d_mean = interpol_and_average(i, '300G dens', tau, d[i, :, sample].T, grid, Ns)

    Ti_300[i, : len(T_mean)] = T_mean
    pi_300[i, : len(p_mean)] = p_mean
    di_300[i, : len(d_mean)] = d_mean
    
np.savez('intro_img_interpol_tpd', grid_ssd = grid_ssd, Ti_ssd = Ti_ssd, pi_ssd = pi_ssd, di_ssd = di_ssd,\
                                   grid_300 = grid_300, Ti_300 = Ti_300, pi_300 = pi_300, di_300 = di_300)

---------------
mu =  10
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...
reading pressure...
reading density...


interpolate and average SSD dens: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:48<00:00, 5380.52it/s]


300G cube
reading tau...
sampling tau...
reading temperature...
reading pressure...
reading density...


interpolate and average 300G dens: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:49<00:00, 5289.93it/s]


---------------
mu =  8
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...
reading pressure...
reading density...


...

interpolate and average 300G dens: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:50<00:00, 5238.77it/s]


---------------
mu =  2
---------------
SSD cube
reading tau...
sampling tau...
reading temperature...
reading pressure...
reading density...


interpolate and average SSD dens: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:48<00:00, 5371.81it/s]


300G cube
reading tau...
sampling tau...
reading temperature...
reading pressure...
reading density...


interpolate and average 300G dens: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [00:50<00:00, 5220.17it/s]


In [2]:
grid_ssd = np.load('intro_img_interpol_tpd.npz')['grid_ssd']
grid_300 = np.load('intro_img_interpol_tpd.npz')['grid_300']

Ti_ssd =   np.load('intro_img_interpol_tpd.npz')['Ti_ssd']
Ti_300 =   np.load('intro_img_interpol_tpd.npz')['Ti_300']

pi_ssd =   np.load('intro_img_interpol_tpd.npz')['pi_ssd']
pi_300 =   np.load('intro_img_interpol_tpd.npz')['pi_300']

di_ssd =   np.load('intro_img_interpol_tpd.npz')['di_ssd']
di_300 =   np.load('intro_img_interpol_tpd.npz')['di_300']

In [17]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

from tqdm import tqdm

from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.8', '0.6', '0.4', '0.2']

idx_mu = [0, 2, 4]

fig = plt.figure(dpi = 300)

fig.set_size_inches(10, 11.25 * 2 / 5, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(2, 3, width_ratios = [1, 1, 1], hspace = 0.2, wspace = 0.3)

tem_ssd = plt.subplot(gs[0, 0])
pre_ssd = plt.subplot(gs[0, 1])
den_ssd = plt.subplot(gs[0, 2])

tem_300 = plt.subplot(gs[1, 0])
pre_300 = plt.subplot(gs[1, 1])
den_300 = plt.subplot(gs[1, 2])

for i in range(len(idx_mu)):

    tem_ssd.plot(grid_ssd[idx_mu[i], :], Ti_ssd[idx_mu[i], :], label = r'$\mu = $' + mus[idx_mu[i]])
    pre_ssd.plot(grid_ssd[idx_mu[i], :], pi_ssd[idx_mu[i], :])
    den_ssd.plot(grid_ssd[idx_mu[i], :], di_ssd[idx_mu[i], :])
    
    tem_300.plot(grid_300[idx_mu[i], :], Ti_300[idx_mu[i], :])
    pre_300.plot(grid_300[idx_mu[i], :], pi_300[idx_mu[i], :])
    den_300.plot(grid_300[idx_mu[i], :], di_300[idx_mu[i], :])
    
    den2_ssd = den_ssd.twinx()
    den2_300 = den_300.twinx()
    
    den2_ssd.yaxis.set_ticks([])
    den2_300.yaxis.set_ticks([])

    tem_ssd.set_xscale('log')
    pre_ssd.set_xscale('log')
    den_ssd.set_xscale('log')
    tem_300.set_xscale('log')
    pre_300.set_xscale('log')
    den_300.set_xscale('log')
    
    pre_ssd.set_yscale('log')
    den_ssd.set_yscale('log')
    pre_300.set_yscale('log')
    den_300.set_yscale('log')

    tem_ssd.set_xlim(3e+1, 2e-10)
    pre_ssd.set_xlim(tem_ssd.get_xlim())
    den_ssd.set_xlim(tem_ssd.get_xlim())
    tem_300.set_xlim(tem_ssd.get_xlim())
    pre_300.set_xlim(tem_ssd.get_xlim())
    den_300.set_xlim(tem_ssd.get_xlim())
    
    tem_ssd.set_ylim(3000, 10000)
    tem_300.set_ylim(3000, 10000)

    tem_ssd.yaxis.set_minor_locator(AutoMinorLocator(4))
    tem_300.yaxis.set_minor_locator(AutoMinorLocator(4))

    tem_ssd.set_ylabel('Temperature, K')
    tem_300.set_ylabel('Temperature, K')
    
    pre_ssd.set_ylabel('Pressure, g / cm / s$^2$')
    pre_300.set_ylabel('Pressure, g / cm / s$^2$')

    den_ssd.set_ylabel(r'Density, g / cm / s$^2$')
    den_300.set_ylabel(r'Density, g / cm / s$^2$')
    
    den2_ssd.set_ylabel('SSD')
    den2_300.set_ylabel('300G')
    
    tem_300.set_xlabel(r'$\tau_\mathrm{Ross}$')
    pre_300.set_xlabel(r'$\tau_\mathrm{Ross}$')
    den_300.set_xlabel(r'$\tau_\mathrm{Ross}$')
    
    tem_ssd.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})

plt.savefig('intro_img_2_tpd.pdf', bbox_inches = 'tight')

plt.close('all')